In [42]:
import pandas as pd
import numpy as np
from os.path import exists
from os import mkdir

In [3]:
binary_table = pd.read_table('~/Desktop/StringData/stringdata/hpo_anno_samples.tsv',sep='\t',index_col=0)
binary_table.iloc[:10,:]

,HP:0000002,HP:0000003,HP:0000008,HP:0000009,HP:0000010,HP:0000011,HP:0000012,HP:0000013,HP:0000014,HP:0000015,...,HP:0410030,HP:0410031,HP:0410042,HP:0410043,HP:0410067,HP:0500012,HP:0500015,HP:3000030,HP:3000036,HP:3000050
A0PJY2,0,0,1,0,0,0,0,0,0,0,...,0,0,-1,0,-1,1,0,0,0,0
A1X283,0,-1,-1,-1,0,-1,-1,-1,-1,-1,...,0,0,0,0,-1,-1,0,0,0,0
A2IDD5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,0,-1,-1,-1,-1,-1,-1
A2PYH4,-1,0,1,0,0,0,0,0,0,0,...,-1,-1,-1,0,-1,0,-1,-1,-1,-1
A2RRP1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,1,0,0,-1,0,0,0,0
A2RUG3,-1,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
A4D126,-1,0,0,0,0,0,0,0,0,0,...,0,1,-1,0,0,-1,-1,0,0,0
A5D8V7,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,...,0,0,-1,-1,-1,-1,0,0,0,-1
A6NCS4,0,0,0,0,0,0,0,0,0,0,...,0,0,-1,-1,0,-1,0,0,0,0
A6NFK2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [61]:
features=['neighborhood_transferred',
 'fusion',
 'cooccurence',
 'homology',
 'coexpression',
 'coexpression_transferred',
 'experiments',
 'experiments_transferred',
 'database',
 'database_transferred',
 'textmining',
 'textmining_transferred',
 'combined_score']

In [ ]:
for term in binary_table.columns:
    t = term[:2] + term[3:]
    if not exists('/Users/wangl35/Desktop/StringData/hpo/%s' %t):
        mkdir('/Users/wangl35/Desktop/StringData/hpo/%s' %t)
    binary_term = binary_table[term]
    binary_term = binary_term.loc[binary_term != 0]
    term_inds = binary_term.index
    
    for feature in features:
        feature_df =  pd.read_csv('/Users/wangl35/Desktop/StringData/stringdata/%s.csv'%feature,index_col=0)
        inds = [ind for ind in feature_df.index.tolist() if ind in term_inds]
        feature_df = feature_df.loc[inds,]
        binary_term = binary_term.loc[inds]
        feature_df['cls'] = binary_term
        del feature_df.index.name
#         feature_df.astype(str)
        convert_to_arff(feature_df,'/Users/wangl35/Desktop/StringData/hpo/%s' %t, feature)
        #feature_df.to_csv('~/Desktop/StringData/hpo/%s/%s.csv' %(term,feature))
        print term, feature

HP:0000002 neighborhood_transferred
HP:0000002 fusion
HP:0000002 cooccurence
HP:0000002 homology
HP:0000002 coexpression
HP:0000002 coexpression_transferred
HP:0000002 experiments
HP:0000002 experiments_transferred
HP:0000002 database
HP:0000002 database_transferred
HP:0000002 textmining
HP:0000002 textmining_transferred
HP:0000002 combined_score
HP:0000003 neighborhood_transferred
HP:0000003 fusion
HP:0000003 cooccurence


In [63]:
def convert_to_arff(df,path,name):
    fn = open('%s/%s.arff' %(path,name), 'w')
    fn.write('@relation linhuaw_%s\n'%name)
    for col in df.columns:
        if col != 'cls':
            fn.write('@attribute %s numeric\n' %col)
        else:
            fn.write('@attribute col {-1,1}\n')
    fn.write('@data\n')
    for i in range(df.shape[0]):
        fn.write(','.join(map(str,df.values[i])))
        fn.write('\n')
    fn.close()